In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import argparse
import geopandas as gpd
import numpy as np
import matplotlib.colors as colors
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D
import tkinter as tk
from tkinter.simpledialog import askinteger
import os
from copy import deepcopy
import threading
from tkinter import simpledialog
import random

from pathlib import Path
from copy import deepcopy
# from cjio import cityjson
from shapely.geometry import Polygon,Point
import matplotlib.pyplot as plt
plt.close('all')
# from sklearn.preprocessing import FunctionTransformer
# from sklearn import cluster
import numpy as np
import shapely


In [6]:
geojson_file = "denhaag_roof.geojson"
gdf = gpd.read_file(geojson_file)
gdf['centroid'] = gdf.centroid

# for column_name in gdf.columns:

In [ ]:
attributes = '''id,
             intersection_angle_max_min,
             largest_polygon_area,
             second_largest_polygon_area,
             second_largest_degree,
             ground_degree_avg,
             ground_degree_std,
             ground_degree_median,
             Min_Area_Proportion,
             Absolute_Max_Height (roof to ground),
             Relative_Max_Height (Maximum roof height - minimum), 
             
             Roof_height_standard_deviation, 
             Roof_height_median, 
             The number of small roof parts,  
             The area proportion of small roof parts,   
             
             the squareness_avg, 
             the squareness_med, 
             the squareness_std'''
#the number of horizontal roofs, 
attributes = {attribute.strip():[] for attribute in attributes.split(",")}
print(attributes)
for geom in gdf.values:
        ids,multi_polygon = geom[0],geom[-2]
        print("id is :",ids)
        attributes["id"].append(ids)
        areas = {i: poly.area for i, poly in enumerate(multi_polygon)}
        ground_normal = np.array([0, 0, 1])
        

        # calculate the normal vector of the polygon
        def calculate_normal(coords):
            p1, p2, p3 = coords[:3]
            
            v1 = np.array([p2[0] - p1[0], p2[1] - p1[1],p2[2] - p1[2]])
            v2 = np.array([p3[0] - p1[0], p3[1] - p1[1],p2[2] - p1[2]])
            normal = np.cross(v1, v2)
            length = np.linalg.norm(normal)

            # normalize the normal vector
            normalized_vector = normal / length
            return normalized_vector

        
        def calculate_degree(vector1,vector2):
            angle_rad = np.arccos(np.dot(vector1, vector2))

            # convert radians to degrees
            angle_deg = abs(np.degrees(angle_rad))
            if abs(angle_deg) <4:
                angle_deg =0
            elif abs(angle_deg-90) <4:
                angle_deg = 90
            elif abs(angle_deg-180) <4:
                angle_deg = 0
            elif abs(angle_deg-270) <4:
                angle_deg = 90
            elif abs(angle_deg-360) <4:
                angle_deg = 0
            return angle_deg
        
        def calculate_centroid(polygon):
            vertices = list(polygon.exterior.coords)

            # calculate the number of vertices
            n = len(vertices)

            # initialize coordinate sums
            sum_x, sum_y, sum_z = 0, 0, 0

            # calculate coordinate sums
            for x, y, z in vertices:
                sum_x += x
                sum_y += y
                sum_z += z

            # calculate centroid using coordinate sums
            centroid_x = sum_x / n
            centroid_y = sum_y / n
            centroid_z = sum_z / n
            return Point(centroid_x,centroid_y,centroid_z)
        
        def calculate_squareness(polygon):
            bounds = polygon.bounds
            width = bounds[2] - bounds[0]
            height = bounds[3] - bounds[1]

            # Calculate the aspect ratio (squareness)
            aspect_ratio = max(width, height) / min(width, height)

            return aspect_ratio
        
        sorted_areas2 = sorted(areas.items(), key=lambda x: x[1], reverse=True)
        sorted_areas = np.array([poly.area for i, poly in enumerate(multi_polygon)])
        print(sorted_areas)
        polygons = [multi_polygon[index] for index in range(len(multi_polygon))]
        
        squareness = np.array([calculate_squareness(poly) for poly in polygons])
        attributes["the squareness_avg"].append(squareness.mean()) 
        attributes["the squareness_std"].append(squareness.std()) 
        attributes["the squareness_med"].append(np.median(squareness))
        
        # calculate normal vectors for all polygons
        coords = [list(polygon.exterior.coords) for polygon in multi_polygon]
    
        vectors = [calculate_normal(coord) for coord in coords]
        vectors_ground = np.array([calculate_degree(vector,ground_normal) for vector in vectors ])
        attributes["ground_degree_avg"].append(vectors_ground.mean())
        attributes["ground_degree_std"].append(vectors_ground.std())
        attributes["ground_degree_median"].append(np.median(vectors_ground))
        #attributes["the number of horizontal roofs"].append(sum(vectors_ground==0))


        attributes["Min_Area_Proportion"].append(min(sorted_areas)/sum(sorted_areas))
            
        
        height = np.array([calculate_centroid(poly).z for poly in polygons])
        height_avg = height.mean()
        height_std = height.std()
        height_median = np.median(height)
        attributes["Roof_height_median"].append(height_median)
        attributes["Roof_height_standard_deviation"].append(height_std)
        attributes["The number of small roof parts"].append(sum(sorted_areas<5))
    
        
        attributes["The area proportion of small roof parts"].append(sum(sorted_areas[sorted_areas<5])/sum(sorted_areas))
        attributes["Absolute_Max_Height (roof to ground)"].append(max(height))
        attributes["Relative_Max_Height (Maximum roof height - minimum)"].append(max(height)-min(height))
        
        
        
        
        if len(sorted_areas) == 1:
            attributes["intersection_angle_max_min"].append(0)
            attributes["largest_polygon_area"].append(areas[sorted_areas2[0][0]])
            #print("max areas",areas[sorted_polygons[0][0]])
            attributes["second_largest_polygon_area"].append(0)

            attributes["second_largest_degree"].append(0)
        
        else:
            largest_polygon_index = sorted_areas2[0][0]
            second_largest_polygon_index = sorted_areas2[1][0]

            # get the largest and second largest polygon
            largest_polygon = multi_polygon[largest_polygon_index]
            second_largest_polygon = multi_polygon[second_largest_polygon_index]
            attributes["largest_polygon_area"].append(areas[sorted_areas2[0][0]])
            attributes["second_largest_polygon_area"].append(areas[sorted_areas2[1][0]])

            # calculate the normal vector of the polygon
            largest_coords = list(largest_polygon.exterior.coords)
            second_largest_coords = list(second_largest_polygon.exterior.coords)
            vector1 = calculate_normal(largest_coords)
            vector2 = calculate_normal(second_largest_coords)
            # calculate the intersection angle between the two polygons

            attributes["intersection_angle_max_min"].append(calculate_degree(vector1,vector2)) 

            attributes["second_largest_degree"].append(calculate_degree(vector2,ground_normal)) 

        

In [8]:
import pandas as pd
dataframe = pd.DataFrame(attributes)
dataframe.head()

# Assuming 'dataframe' is a DataFrame containing your data
with open("denhaag_roof_new_a.csv", "w", newline="\n") as file:
    dataframe.to_csv(file)


In [42]:
import pandas as pd

# clean the first roof feature CSV file
# Load the first roof feature CSV file into a DataFrame
df_old = pd.read_csv("denhaag_roof_old.csv")

# Get a list of column names to drop
columns_to_drop = [col for col in df_old.columns if col.startswith("Unnamed:")]

# Drop the specified columns
df_old = df_old.drop(columns=columns_to_drop)

# Save the DataFrame to a new CSV file
df_old.to_csv("denhaag_roof_old_cleaned.csv")

#print(df_old.head())
#print(df_old.columns)
#df_new = pd.read_csv("denhaag_roof_new.csv")

C:\Users\cheng\AppData\Local\Temp\ipykernel_14504\1115074889.py:4: DtypeWarning: Columns (0,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

In [9]:
#merge two csv files
import pandas as pd

# Load the first roof feature and the second roof feature CSV files into DataFrames
df_old = pd.read_csv("denhaag_roof_old_horzontial.csv")

df_new = pd.read_csv("denhaag_roof_new_a.csv")

# Merge the DataFrames based on the common columns "id" and "ID"
merged_df = pd.merge(df_old, df_new, left_on="Id", right_on="id")

# Drop one of the duplicate columns (e.g., "Id" or "id") if needed
merged_df.drop("id", axis=1, inplace=True)

columns_to_drop = [col for col in merged_df.columns if col.startswith("Unnamed:")]
print(columns_to_drop)
# Drop the specified columns
merged_df_clean = merged_df.drop(columns=columns_to_drop)

# Save the merged DataFrame to a new CSV file
merged_df_clean.to_csv("merged_denhaag_roof_features_a.csv")


['Unnamed: 0_x', 'Unnamed: 0_y']
